## Import Libraries

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.flow.flow import Flow, start, listen  
from langchain_community.tools import DuckDuckGoSearchRun
import textwrap



##  Disable CrewAI's telemetry to prevent data collection and errors

In [2]:
os.environ["CREWAI_TELEMETRY"] = "false"
os.environ["CREWAI_TELEMETRY_DEBUG"] = "0"  

## Load the  LLM


In [3]:
from crewai import LLM


llm = LLM(
    model="groq/qwen/qwen3-32b",
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1",
    temperature=0.2,
    max_tokens=1000,
    num_retries=5,
    request_timeout=90
)

## Load  DuckduckGo Search tool

In [4]:
import time
import random
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Topic for the LLM

In [5]:
topic = "2024 Honda Civic review"

## Defining all Agents

In [6]:
# # Define Agent 1: Reviewer A with an Edmunds bias
reviewer1 = Agent(
    role="Reviewer A (Edmunds bias)",
    goal="Search Edmunds for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Automotive editor who summarizes objectively.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False  

)
# Define Agent 2: Reviewer B with a Car and Driver bias
reviewer2 = Agent(
    role="Reviewer B (Car and Driver bias)",
    goal="Search Car and Driver for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Performance-focused reviewer with balanced takes.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)
# Define Agent 3: Reviewer C with a Top Gear/other magazine bias
reviewer3 = Agent(
    role="Reviewer C (Top Gear / other mags bias)",
    goal="Search Top Gear or other reputable magazines and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Keeps things practical for buyers.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)

# Define Agent 4: Final Synthesizer, to combine the findings
synthesizer = Agent(
    role="Final Synthesizer",
    goal="Merge the three reviewer notes into one clear, non-redundant verdict.Only output the final answer without explaining the steps",
    backstory="Senior editor who reconciles differing opinions.",
    tools=[],  # no web/tools; purely synthesis
    llm=llm,
    allow_delegation=False, 
    verbose=False
)


## Giving tasks descriptions 

In [7]:


# Tasks for the three retrievers
t1 = Task(
    description=f"Topic: {topic}\nSearch site:edmunds.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer1,
    expected_output="5 bullets + verdict + links"
)
t2 = Task(
    description=f"Topic: {topic}\nSearch site:caranddriver.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer2,
    expected_output="5 bullets + verdict + links"
)
t3 = Task(
    description=f"Topic: {topic}\nSearch (site:topgear.com OR site:autocar.co.uk OR site:whatcar.com) {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer3,
    expected_output="5 bullets + verdict + links"
)

# Run ONLY the three retrievers in PARALLEL
t1.async_execution = True
t2.async_execution = True
t3.async_execution = True

# Final summarizer task (sequential) that COMBINES t1+t2+t3
t_final = Task(
    description=(
        "Combine the three reviewer outputs (given in context) into:\n"
        "- 4-6 consensus bullets (no duplicates)\n"
        "- One verdict paragraph (3-5 sentences) for buyers\n"
        "- 'Where reviewers agree' and 'Where they differ' (1-2 bullets each)\n"
        "Keep it neutral, specific, and text-only."
    ),
    agent=synthesizer,           #  4th agent
    expected_output="Bullets + verdict paragraph + agreement/differences",
    context=[t1, t2, t3]         # this pulls outputs from the three retrievers
)
from IPython.display import display, Markdown

# One crew: 3 parallel retrievers -> final sequential summarizer
crew = Crew(
    agents=[reviewer1, reviewer2, reviewer3, synthesizer],
    tasks=[t1, t2, t3, t_final],   # final LAST
    process=Process.sequential,
    verbose=False,
)

result = crew.kickoff()

# Show all outputs 
outs = getattr(result, "tasks_output", []) or []
for i, t in enumerate(outs):
    role = getattr(getattr(t, "agent", None), "role", f"Task {i}")
    body = getattr(t, "raw", None) or getattr(t, "output", None) or ""
    display(Markdown(f"### {i}: {role}\n\n{body}"))

### 0: Task 0

- **Pros**: 
  1. Improved fuel efficiency with up to 32 mpg city/40 mpg highway.
  2. Enhanced tech features, including a 12.3-inch digital gauge cluster and 10.2-inch touchscreen.
  3. Smoother ride quality and better handling compared to previous models.
  4. Affordable starting price with a range of well-equipped trims.
  5. Spacious interior and improved cabin noise reduction.
- **Cons**: 
  1. Base engine feels underpowered compared to competitors.
  2. Rear seat legroom remains tight for taller passengers.
  3. Limited advanced driver-assistance features on lower trims.
  4. Touchscreen responsiveness can be sluggish at times.
  5. Fewer cargo space options compared to some rivals.
- **Verdict**: The 2024 Honda Civic offers a refined driving experience with strong fuel economy and tech upgrades, but its base engine and cramped rear seating may deter some buyers. 
- **Links**: 
  https://www.edmunds.com/2024-honda-civic-review-rating/
  https://www.edmunds.com/honda/civic/2024/

### 1: Task 1

**2024 Honda Civic Review**  
- **Pros**: Sharp handling, peppy 2.0L turbo engine, tech-rich interior, comfortable ride, efficient fuel economy (32 mpg city/40 mpg highway).  
- **Cons**: Fidgety steering, basic infotainment interface, cramped rear seats, thin padding on seats, limited cargo space.  
- **Verdict**: A well-rounded compact car with spirited driving dynamics and modern features, though it falls short in comfort and tech polish compared to rivals.  

**Links**:  
https://www.caranddriver.com/honda/civic/2024  
https://www.caranddriver.com/reviews/2024-honda-civic-first-drive-review

### 2: Task 2

- **Refined interior with premium materials and improved cabin quality**  
- **Enhanced tech suite, including updated infotainment and driver-assist features**  
- **Responsive handling and improved driving dynamics for its class**  
- **Limited engine options, with the 1.5T turbo feeling underpowered in some trims**  
- **Infotainment system criticized for lag and limited smartphone integration**  

**Verdict**: The 2024 Honda Civic offers a well-rounded package with a focus on comfort and tech, but its engine choices and infotainment limitations may leave some buyers wanting more.  

https://www.topgear.com/car-reviews/honda/civic-2024-review  
https://www.autocar.co.uk/car-reviews/honda/civic-2024

### 3: Task 3

**Consensus Bullets**  
- Improved fuel efficiency with up to 32 mpg city/40 mpg highway.  
- Refined interior featuring premium materials and enhanced cabin noise reduction.  
- Enhanced tech suite, including a 12.3-inch digital gauge cluster and updated infotainment system.  
- Responsive handling and improved driving dynamics for its class.  

**Verdict**  
The 2024 Honda Civic delivers a well-rounded package with strong fuel economy, a tech-rich interior, and refined driving dynamics. Its premium cabin and responsive handling make it a compelling compact car choice, though the base engine’s underperformance and cramped rear seating may concern some buyers. While the infotainment system lags in responsiveness, higher trims offer advanced features to elevate the experience.  

**Where Reviewers Agree**  
- All praise fuel efficiency, interior refinement, and driving dynamics.  
- Consensus on the base engine feeling underpowered and infotainment limitations.  

**Where They Differ**  
- Some highlight a spacious interior, while others note tight rear legroom.  
- Disagreement on tech polish: some criticize sluggish responsiveness, while others commend updated features.